In [48]:
import toolz as tz
import operator as op
import inspect


In [63]:
def repr(obj):
    if callable(obj):
        return f'{obj.__name__}{inspect.signature(obj)}'

    return obj

In [73]:
class Maybe:
    def __init__(self, x):
        self._value = x
        self.isNothing = self._value is None

    @classmethod
    def of(cls, x):
        return cls(x)

    def map(self, fn):
        return self if self.isNothing else Maybe.of(fn(self._value))

    def join(self):
        return Maybe.of(None) if self.isNothing else self._value

    def chain(self, fn):
        return self.map(fn).join()

    def ap(self, f):
        return self if self.isNothing else f.map(self._value)

    def __repr__(self):
        return "Nothing" if self.isNothing else f"Just({repr(self._value)})"


class Either:
    def __init__(self, x):
        self._value = x

    @classmethod
    def of(cls, x):
        return Right(x)


class Right(Either):
    def map(self, fn):
        return Either.of(fn(self._value))

    def join(self):
        return self._value

    def chain(self, fn):
        return fn(self._value)

    def __repr__(self):
        return f"Right({repr(self._value)})"


class Left(Either):
    def map(self):
        return self

    def join(self):
        return self

    def chain(self):
        return self

    def ap(self):
        return self

    def __repr__(self):
        return f"Left({repr(self._value)})"


class IO:
    def __init__(self, io):
        self.unsafePerformIO = io

    @classmethod
    def of(cls, x):
        return cls(lambda: x)

    def map(self, fn):
        return IO(tz.compose(fn, self.unsafePerformIO))

    def join(self):
        return IO.of(self.unsafePerformIO().unsafePerformIO())

    def chain(self, fn):
        return self.map(fn).join()

    def ap(self,f):
        return self.chain(lambda fn: f.map(fn))

    def __repr__(self):
        return f"IO({repr(self.unsafePerformIO)})"


In [75]:
@tz.curry
def prop(name, obj):
    return obj[name]


@tz.curry
def map_(f, functor):
    return functor.map(f)


map_iter = tz.curry(map)


@tz.curry
def maybe(v, f, m):
    return v if m.isNothing else f(m.__value__)


left = lambda x: Left(x)


@tz.curry
def either(f, g, e):
    if isinstance(e, Left):
        return f(e.__value__)
    elif isinstance(e, Right):
        return g(e.__value__)

    return None


@tz.curry
def safeProp(key, obj):
    try:
        return Maybe.of(obj[key])
    except Exception as e:
        return Maybe(None)


def safeHead(lst):
    try:
        return Maybe.of(lst[0])
    except Exception as e:
        return Maybe.of(None)


# join :: Monad m => m (m a) -> m a
join = lambda mma: mma.join()

# chain :: Monad m => (a -> m b) -> m a -> m b
chain = tz.curry(lambda f, m: m.map(f).join())

add = tz.curry(op.add)


In [79]:
Maybe.of(2).map(add).ap(Maybe.of(3)), \
    Maybe.of(add).ap(Maybe.of(2)).ap(Either.of(3)), \
    IO.of(add).ap(IO.of(2)).ap(IO.of(3))

(Just(5), Right(5), IO(<lambda>()))